In [15]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import pyautogui  # ใช้สำหรับควบคุมวิดีโอ
import time

# ตั้งค่ากล้อง
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8)  # เพิ่มความแม่นยำในการตรวจจับ

# ตั้งค่าระบบเสียง
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# กำหนดค่าต่ำสุด-สูงสุดของระยะมือ
min_distance = 20
max_distance = 180
adjusting_volume = False  # เช็คว่ากำลังปรับเสียงอยู่หรือไม่
last_volume = volume.GetMasterVolumeLevelScalar()  # เก็บระดับเสียงล่าสุด

# ตัวแปรใหม่เช็คสถานะการกด
last_action = None
last_time = time.time()  # ใช้จับเวลาเพื่อป้องกันการกดซ้ำ
last_move_time = time.time()  # จับเวลาสำหรับการเลื่อนคลิป
last_skip_time = time.time()  # สำหรับจับเวลาเมื่อข้ามโฆษณา
last_next_time = time.time()  # สำหรับจับเวลาไปคลิปถัดไป

# ความเร็วในการเลื่อนคลิป
move_interval = 1  # 1 วินาทีในการเลื่อนคลิป
move_duration = 5  # 5 วินาทีในการเลื่อนคลิป
skip_interval = 5  # 5 วินาทีหลังจากข้ามโฆษณา
next_interval = 5  # 5 วินาทีหลังจากไปคลิปถัดไป

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)

    if hands:
        for hand in hands:
            fingers = detector.fingersUp(hand)  # ตรวจสอบนิ้วที่ยก

            if hand["type"] == "Right" and len(hand["lmList"]) >= 9:  # มือขวา -> ควบคุมเสียง
                if fingers == [1, 1, 1, 1, 1]:  # กางมือ 5 นิ้ว -> หยุดที่ระดับเสียงเดิม
                    adjusting_volume = False  

                elif fingers == [1, 1, 0, 0, 0]:  # ชูแค่โป้งกับชี้ -> ปรับเสียง
                    adjusting_volume = True
                    thumb_tip = hand["lmList"][4]
                    index_tip = hand["lmList"][8]
                    
                    # คำนวณระยะห่างระหว่างนิ้วโป้งและนิ้วชี้
                    distance_x = abs(thumb_tip[0] - index_tip[0])
                    distance_y = abs(thumb_tip[1] - index_tip[1])
                    distance = math.hypot(distance_x, distance_y)
                    distance = max(min_distance, min(distance, max_distance))

                    volume_scalar = (distance - min_distance) / (max_distance - min_distance)

                    # ทำให้การเปลี่ยนเสียงนุ่มนวลขึ้น
                    smooth_factor = 0.2  
                    last_volume = (1 - smooth_factor) * last_volume + smooth_factor * volume_scalar

                    # ป้องกันการเปลี่ยนเสียงถ้าต่างกันน้อยเกินไป
                    if abs(volume.GetMasterVolumeLevelScalar() - last_volume) > 0.02:
                        volume.SetMasterVolumeLevelScalar(last_volume, None)

                    # วาดเส้นระหว่างนิ้วโป้งกับชี้
                    cv2.line(img, (thumb_tip[0], thumb_tip[1]), (index_tip[0], index_tip[1]), (0, 255, 0), 3)

                # แสดงระดับเสียง
                volume_percent = int(last_volume * 100)
                cv2.putText(img, f'Volume: {volume_percent}%', (50, 50),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

                # แสดงแถบระดับเสียง
                bar_height = int((1 - last_volume) * 300)
                bar_color = (0, 255, 0) if last_volume > 0.5 else (0, 0, 255)  # สีเขียว = ดัง, แดง = เบา
                cv2.rectangle(img, (50, 100), (100, 400), (255, 0, 0), 3)  
                cv2.rectangle(img, (50, 100 + bar_height), (100, 400), bar_color, -1)

                # แสดงไอคอนเสียง
                cv2.putText(img, "🔊" if last_volume > 0.5 else "🔈", (120, 380), 
                            cv2.FONT_HERSHEY_COMPLEX, 1, bar_color, 2)

            elif hand["type"] == "Left" and len(hand["lmList"]) >= 9:  # มือซ้าย -> ควบคุมวิดีโอ
                current_time = time.time()

                if current_time - last_time > 1:  # เช็คเวลาห่างกัน 1 วินาที
                    if fingers == [0, 1, 0, 0, 0]:  # ชี้นิ้วมือซ้ายไปทางขวา (เลื่อนคลิปไปข้างหน้า 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('right')  # เลื่อนคลิปไปข้างหน้า 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน

                    elif fingers == [0, 1, 1, 0, 0]:  # ชี้ 2 นิ้ว (นิ้วชี้, นิ้วกลาง) (ย้อนคลิป 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('left')  # เลื่อนคลิปย้อนกลับ 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน


                    # ไปคลิปถัดไป
                    elif fingers == [0, 1, 1, 1, 1]:  # ชี้ 4 นิ้ว (ชี้นิ้ว, นิ้วกลาง, นิ้วนาง, นิ้วก้อย)
                        if current_time - last_next_time > next_interval:  # 5 วินาที
                            pyautogui.hotkey('shift', 'n')  # ใช้ shift + n ไปคลิปถัดไป
                            last_next_time = current_time  # รีเซ็ตเวลาการไปคลิปถัดไป


                    elif fingers == [0, 0, 0, 0, 1]:  # กำมือ = หยุดวิดีโอ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

                    elif fingers == [1, 1, 1, 1, 1]:  # แบมือ 5 นิ้ว = เล่นวิดีโอต่อ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

    img = cv2.resize(img, (960, 720))
    cv2.imshow("image", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
 

ModuleNotFoundError: No module named 'cvzone'

In [5]:
!pip install --upgrade opencv-python opencv-contrib-python



  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl (46.2 MB)


In [7]:
import sys
print(sys.executable)



C:\Users\ravip\anaconda3\python.exe


In [9]:
import sys
!{sys.executable} -m pip install --upgrade opencv-python opencv-contrib-python


In [11]:
import cv2
print(cv2.__version__)


4.11.0


In [25]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import pyautogui  # ใช้สำหรับควบคุมวิดีโอ
import time

# ตั้งค่ากล้อง
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8)  # เพิ่มความแม่นยำในการตรวจจับ

# ตั้งค่าระบบเสียง
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# กำหนดค่าต่ำสุด-สูงสุดของระยะมือ
min_distance = 20
max_distance = 180
adjusting_volume = False  # เช็คว่ากำลังปรับเสียงอยู่หรือไม่
last_volume = volume.GetMasterVolumeLevelScalar()  # เก็บระดับเสียงล่าสุด

# ตัวแปรใหม่เช็คสถานะการกด
last_action = None
last_time = time.time()  # ใช้จับเวลาเพื่อป้องกันการกดซ้ำ
last_move_time = time.time()  # จับเวลาสำหรับการเลื่อนคลิป
last_skip_time = time.time()  # สำหรับจับเวลาเมื่อข้ามโฆษณา
last_next_time = time.time()  # สำหรับจับเวลาไปคลิปถัดไป

# ความเร็วในการเลื่อนคลิป
move_interval = 1  # 1 วินาทีในการเลื่อนคลิป
move_duration = 5  # 5 วินาทีในการเลื่อนคลิป
skip_interval = 5  # 5 วินาทีหลังจากข้ามโฆษณา
next_interval = 5  # 5 วินาทีหลังจากไปคลิปถัดไป

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)

    if hands:
        for hand in hands:
            fingers = detector.fingersUp(hand)  # ตรวจสอบนิ้วที่ยก

            if hand["type"] == "Right" and len(hand["lmList"]) >= 9:  # มือขวา -> ควบคุมเสียง
                if fingers == [1, 1, 1, 1, 1]:  # กางมือ 5 นิ้ว -> หยุดที่ระดับเสียงเดิม
                    adjusting_volume = False  

                elif fingers == [1, 1, 0, 0, 0]:  # ชูแค่โป้งกับชี้ -> ปรับเสียง
                    adjusting_volume = True
                    thumb_tip = hand["lmList"][4]
                    index_tip = hand["lmList"][8]
                    
                    # คำนวณระยะห่างระหว่างนิ้วโป้งและนิ้วชี้
                    distance_x = abs(thumb_tip[0] - index_tip[0])
                    distance_y = abs(thumb_tip[1] - index_tip[1])
                    distance = math.hypot(distance_x, distance_y)
                    distance = max(min_distance, min(distance, max_distance))

                    volume_scalar = (distance - min_distance) / (max_distance - min_distance)

                    # ทำให้การเปลี่ยนเสียงนุ่มนวลขึ้น
                    smooth_factor = 0.2  
                    last_volume = (1 - smooth_factor) * last_volume + smooth_factor * volume_scalar

                    # ป้องกันการเปลี่ยนเสียงถ้าต่างกันน้อยเกินไป
                    if abs(volume.GetMasterVolumeLevelScalar() - last_volume) > 0.02:
                        volume.SetMasterVolumeLevelScalar(last_volume, None)

                    # วาดเส้นระหว่างนิ้วโป้งกับชี้
                    cv2.line(img, (thumb_tip[0], thumb_tip[1]), (index_tip[0], index_tip[1]), (0, 255, 0), 3)

                # แสดงระดับเสียง
                volume_percent = int(last_volume * 100)
                cv2.putText(img, f'Volume: {volume_percent}%', (50, 50),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

                # แสดงแถบระดับเสียง
                bar_height = int((1 - last_volume) * 300)
                bar_color = (0, 255, 0) if last_volume > 0.5 else (0, 0, 255)  # สีเขียว = ดัง, แดง = เบา
                cv2.rectangle(img, (50, 100), (100, 400), (255, 0, 0), 3)  
                cv2.rectangle(img, (50, 100 + bar_height), (100, 400), bar_color, -1)

                # แสดงไอคอนเสียง
                cv2.putText(img, "🔊" if last_volume > 0.5 else "🔈", (120, 380), 
                            cv2.FONT_HERSHEY_COMPLEX, 1, bar_color, 2)

            elif hand["type"] == "Left" and len(hand["lmList"]) >= 9:  # มือซ้าย -> ควบคุมวิดีโอ
                current_time = time.time()

                if current_time - last_time > 1:  # เช็คเวลาห่างกัน 1 วินาที
                    if fingers == [0, 1, 0, 0, 0]:  # ชี้นิ้วมือซ้ายไปทางขวา (เลื่อนคลิปไปข้างหน้า 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('right')  # เลื่อนคลิปไปข้างหน้า 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน

                    elif fingers == [0, 1, 1, 0, 0]:  # ชี้ 2 นิ้ว (นิ้วชี้, นิ้วกลาง) (ย้อนคลิป 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('left')  # เลื่อนคลิปย้อนกลับ 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน


                    # ไปคลิปถัดไป
                    elif fingers == [0, 1, 1, 1, 1]:  # ชี้ 4 นิ้ว (ชี้นิ้ว, นิ้วกลาง, นิ้วนาง, นิ้วก้อย)
                        if current_time - last_next_time > next_interval:  # 5 วินาที
                            pyautogui.hotkey('shift', 'n')  # ใช้ shift + n ไปคลิปถัดไป
                            last_next_time = current_time  # รีเซ็ตเวลาการไปคลิปถัดไป


                    elif fingers == [0, 0, 0, 0, 1]:  # กำมือ = หยุดวิดีโอ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

                    elif fingers == [1, 1, 1, 1, 1]:  # แบมือ 5 นิ้ว = เล่นวิดีโอต่อ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

    img = cv2.resize(img, (960, 720))
    cv2.imshow("image", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
 

ModuleNotFoundError: No module named 'cvzone'

In [27]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import pyautogui  # ใช้สำหรับควบคุมวิดีโอ
import time

# ตั้งค่ากล้อง
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8)  # เพิ่มความแม่นยำในการตรวจจับ

# ตั้งค่าระบบเสียง
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# กำหนดค่าต่ำสุด-สูงสุดของระยะมือ
min_distance = 20
max_distance = 180
adjusting_volume = False  # เช็คว่ากำลังปรับเสียงอยู่หรือไม่
last_volume = volume.GetMasterVolumeLevelScalar()  # เก็บระดับเสียงล่าสุด

# ตัวแปรใหม่เช็คสถานะการกด
last_action = None
last_time = time.time()  # ใช้จับเวลาเพื่อป้องกันการกดซ้ำ
last_move_time = time.time()  # จับเวลาสำหรับการเลื่อนคลิป
last_skip_time = time.time()  # สำหรับจับเวลาเมื่อข้ามโฆษณา
last_next_time = time.time()  # สำหรับจับเวลาไปคลิปถัดไป

# ความเร็วในการเลื่อนคลิป
move_interval = 1  # 1 วินาทีในการเลื่อนคลิป
move_duration = 5  # 5 วินาทีในการเลื่อนคลิป
skip_interval = 5  # 5 วินาทีหลังจากข้ามโฆษณา
next_interval = 5  # 5 วินาทีหลังจากไปคลิปถัดไป

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)

    if hands:
        for hand in hands:
            fingers = detector.fingersUp(hand)  # ตรวจสอบนิ้วที่ยก

            if hand["type"] == "Right" and len(hand["lmList"]) >= 9:  # มือขวา -> ควบคุมเสียง
                if fingers == [1, 1, 1, 1, 1]:  # กางมือ 5 นิ้ว -> หยุดที่ระดับเสียงเดิม
                    adjusting_volume = False  

                elif fingers == [1, 1, 0, 0, 0]:  # ชูแค่โป้งกับชี้ -> ปรับเสียง
                    adjusting_volume = True
                    thumb_tip = hand["lmList"][4]
                    index_tip = hand["lmList"][8]
                    
                    # คำนวณระยะห่างระหว่างนิ้วโป้งและนิ้วชี้
                    distance_x = abs(thumb_tip[0] - index_tip[0])
                    distance_y = abs(thumb_tip[1] - index_tip[1])
                    distance = math.hypot(distance_x, distance_y)
                    distance = max(min_distance, min(distance, max_distance))

                    volume_scalar = (distance - min_distance) / (max_distance - min_distance)

                    # ทำให้การเปลี่ยนเสียงนุ่มนวลขึ้น
                    smooth_factor = 0.2  
                    last_volume = (1 - smooth_factor) * last_volume + smooth_factor * volume_scalar

                    # ป้องกันการเปลี่ยนเสียงถ้าต่างกันน้อยเกินไป
                    if abs(volume.GetMasterVolumeLevelScalar() - last_volume) > 0.02:
                        volume.SetMasterVolumeLevelScalar(last_volume, None)

                    # วาดเส้นระหว่างนิ้วโป้งกับชี้
                    cv2.line(img, (thumb_tip[0], thumb_tip[1]), (index_tip[0], index_tip[1]), (0, 255, 0), 3)

                # แสดงระดับเสียง
                volume_percent = int(last_volume * 100)
                cv2.putText(img, f'Volume: {volume_percent}%', (50, 50),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

                # แสดงแถบระดับเสียง
                bar_height = int((1 - last_volume) * 300)
                bar_color = (0, 255, 0) if last_volume > 0.5 else (0, 0, 255)  # สีเขียว = ดัง, แดง = เบา
                cv2.rectangle(img, (50, 100), (100, 400), (255, 0, 0), 3)  
                cv2.rectangle(img, (50, 100 + bar_height), (100, 400), bar_color, -1)

                # แสดงไอคอนเสียง
                cv2.putText(img, "🔊" if last_volume > 0.5 else "🔈", (120, 380), 
                            cv2.FONT_HERSHEY_COMPLEX, 1, bar_color, 2)

            elif hand["type"] == "Left" and len(hand["lmList"]) >= 9:  # มือซ้าย -> ควบคุมวิดีโอ
                current_time = time.time()

                if current_time - last_time > 1:  # เช็คเวลาห่างกัน 1 วินาที
                    if fingers == [0, 1, 0, 0, 0]:  # ชี้นิ้วมือซ้ายไปทางขวา (เลื่อนคลิปไปข้างหน้า 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('right')  # เลื่อนคลิปไปข้างหน้า 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน

                    elif fingers == [0, 1, 1, 0, 0]:  # ชี้ 2 นิ้ว (นิ้วชี้, นิ้วกลาง) (ย้อนคลิป 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('left')  # เลื่อนคลิปย้อนกลับ 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน


                    # ไปคลิปถัดไป
                    elif fingers == [0, 1, 1, 1, 1]:  # ชี้ 4 นิ้ว (ชี้นิ้ว, นิ้วกลาง, นิ้วนาง, นิ้วก้อย)
                        if current_time - last_next_time > next_interval:  # 5 วินาที
                            pyautogui.hotkey('shift', 'n')  # ใช้ shift + n ไปคลิปถัดไป
                            last_next_time = current_time  # รีเซ็ตเวลาการไปคลิปถัดไป


                    elif fingers == [0, 0, 0, 0, 1]:  # กำมือ = หยุดวิดีโอ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

                    elif fingers == [1, 1, 1, 1, 1]:  # แบมือ 5 นิ้ว = เล่นวิดีโอต่อ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

    img = cv2.resize(img, (960, 720))
    cv2.imshow("image", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cvzone'

In [29]:
 import cv2
from cvzone.HandTrackingModule import HandDetector
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import pyautogui  # ใช้สำหรับควบคุมวิดีโอ
import time

# ตั้งค่ากล้อง
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8)  # เพิ่มความแม่นยำในการตรวจจับ

# ตั้งค่าระบบเสียง
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# กำหนดค่าต่ำสุด-สูงสุดของระยะมือ
min_distance = 20
max_distance = 180
adjusting_volume = False  # เช็คว่ากำลังปรับเสียงอยู่หรือไม่
last_volume = volume.GetMasterVolumeLevelScalar()  # เก็บระดับเสียงล่าสุด

# ตัวแปรใหม่เช็คสถานะการกด
last_action = None
last_time = time.time()  # ใช้จับเวลาเพื่อป้องกันการกดซ้ำ
last_move_time = time.time()  # จับเวลาสำหรับการเลื่อนคลิป
last_skip_time = time.time()  # สำหรับจับเวลาเมื่อข้ามโฆษณา
last_next_time = time.time()  # สำหรับจับเวลาไปคลิปถัดไป

# ความเร็วในการเลื่อนคลิป
move_interval = 1  # 1 วินาทีในการเลื่อนคลิป
move_duration = 5  # 5 วินาทีในการเลื่อนคลิป
skip_interval = 5  # 5 วินาทีหลังจากข้ามโฆษณา
next_interval = 5  # 5 วินาทีหลังจากไปคลิปถัดไป

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)

    if hands:
        for hand in hands:
            fingers = detector.fingersUp(hand)  # ตรวจสอบนิ้วที่ยก

            if hand["type"] == "Right" and len(hand["lmList"]) >= 9:  # มือขวา -> ควบคุมเสียง
                if fingers == [1, 1, 1, 1, 1]:  # กางมือ 5 นิ้ว -> หยุดที่ระดับเสียงเดิม
                    adjusting_volume = False  

                elif fingers == [1, 1, 0, 0, 0]:  # ชูแค่โป้งกับชี้ -> ปรับเสียง
                    adjusting_volume = True
                    thumb_tip = hand["lmList"][4]
                    index_tip = hand["lmList"][8]
                    
                    # คำนวณระยะห่างระหว่างนิ้วโป้งและนิ้วชี้
                    distance_x = abs(thumb_tip[0] - index_tip[0])
                    distance_y = abs(thumb_tip[1] - index_tip[1])
                    distance = math.hypot(distance_x, distance_y)
                    distance = max(min_distance, min(distance, max_distance))

                    volume_scalar = (distance - min_distance) / (max_distance - min_distance)

                    # ทำให้การเปลี่ยนเสียงนุ่มนวลขึ้น
                    smooth_factor = 0.2  
                    last_volume = (1 - smooth_factor) * last_volume + smooth_factor * volume_scalar

                    # ป้องกันการเปลี่ยนเสียงถ้าต่างกันน้อยเกินไป
                    if abs(volume.GetMasterVolumeLevelScalar() - last_volume) > 0.02:
                        volume.SetMasterVolumeLevelScalar(last_volume, None)

                    # วาดเส้นระหว่างนิ้วโป้งกับชี้
                    cv2.line(img, (thumb_tip[0], thumb_tip[1]), (index_tip[0], index_tip[1]), (0, 255, 0), 3)

                # แสดงระดับเสียง
                volume_percent = int(last_volume * 100)
                cv2.putText(img, f'Volume: {volume_percent}%', (50, 50),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

                # แสดงแถบระดับเสียง
                bar_height = int((1 - last_volume) * 300)
                bar_color = (0, 255, 0) if last_volume > 0.5 else (0, 0, 255)  # สีเขียว = ดัง, แดง = เบา
                cv2.rectangle(img, (50, 100), (100, 400), (255, 0, 0), 3)  
                cv2.rectangle(img, (50, 100 + bar_height), (100, 400), bar_color, -1)

                # แสดงไอคอนเสียง
                cv2.putText(img, "🔊" if last_volume > 0.5 else "🔈", (120, 380), 
                            cv2.FONT_HERSHEY_COMPLEX, 1, bar_color, 2)

            elif hand["type"] == "Left" and len(hand["lmList"]) >= 9:  # มือซ้าย -> ควบคุมวิดีโอ
                current_time = time.time()

                if current_time - last_time > 1:  # เช็คเวลาห่างกัน 1 วินาที
                    if fingers == [0, 1, 0, 0, 0]:  # ชี้นิ้วมือซ้ายไปทางขวา (เลื่อนคลิปไปข้างหน้า 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('right')  # เลื่อนคลิปไปข้างหน้า 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน

                    elif fingers == [0, 1, 1, 0, 0]:  # ชี้ 2 นิ้ว (นิ้วชี้, นิ้วกลาง) (ย้อนคลิป 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('left')  # เลื่อนคลิปย้อนกลับ 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน


                    # ไปคลิปถัดไป
                    elif fingers == [0, 1, 1, 1, 1]:  # ชี้ 4 นิ้ว (ชี้นิ้ว, นิ้วกลาง, นิ้วนาง, นิ้วก้อย)
                        if current_time - last_next_time > next_interval:  # 5 วินาที
                            pyautogui.hotkey('shift', 'n')  # ใช้ shift + n ไปคลิปถัดไป
                            last_next_time = current_time  # รีเซ็ตเวลาการไปคลิปถัดไป


                    elif fingers == [0, 0, 0, 0, 1]:  # กำมือ = หยุดวิดีโอ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

                    elif fingers == [1, 1, 1, 1, 1]:  # แบมือ 5 นิ้ว = เล่นวิดีโอต่อ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

    img = cv2.resize(img, (960, 720))
    cv2.imshow("image", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cvzone'

In [31]:
import cv2
from cvzone.HandTrackingModule import HandDetector


ModuleNotFoundError: No module named 'mediapipe'

In [35]:
import cv2
from cvzone.HandTrackingModule import HandDetector


In [37]:
 import cv2
from cvzone.HandTrackingModule import HandDetector
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import pyautogui  # ใช้สำหรับควบคุมวิดีโอ
import time

# ตั้งค่ากล้อง
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8)  # เพิ่มความแม่นยำในการตรวจจับ

# ตั้งค่าระบบเสียง
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# กำหนดค่าต่ำสุด-สูงสุดของระยะมือ
min_distance = 20
max_distance = 180
adjusting_volume = False  # เช็คว่ากำลังปรับเสียงอยู่หรือไม่
last_volume = volume.GetMasterVolumeLevelScalar()  # เก็บระดับเสียงล่าสุด

# ตัวแปรใหม่เช็คสถานะการกด
last_action = None
last_time = time.time()  # ใช้จับเวลาเพื่อป้องกันการกดซ้ำ
last_move_time = time.time()  # จับเวลาสำหรับการเลื่อนคลิป
last_skip_time = time.time()  # สำหรับจับเวลาเมื่อข้ามโฆษณา
last_next_time = time.time()  # สำหรับจับเวลาไปคลิปถัดไป

# ความเร็วในการเลื่อนคลิป
move_interval = 1  # 1 วินาทีในการเลื่อนคลิป
move_duration = 5  # 5 วินาทีในการเลื่อนคลิป
skip_interval = 5  # 5 วินาทีหลังจากข้ามโฆษณา
next_interval = 5  # 5 วินาทีหลังจากไปคลิปถัดไป

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)

    if hands:
        for hand in hands:
            fingers = detector.fingersUp(hand)  # ตรวจสอบนิ้วที่ยก

            if hand["type"] == "Right" and len(hand["lmList"]) >= 9:  # มือขวา -> ควบคุมเสียง
                if fingers == [1, 1, 1, 1, 1]:  # กางมือ 5 นิ้ว -> หยุดที่ระดับเสียงเดิม
                    adjusting_volume = False  

                elif fingers == [1, 1, 0, 0, 0]:  # ชูแค่โป้งกับชี้ -> ปรับเสียง
                    adjusting_volume = True
                    thumb_tip = hand["lmList"][4]
                    index_tip = hand["lmList"][8]
                    
                    # คำนวณระยะห่างระหว่างนิ้วโป้งและนิ้วชี้
                    distance_x = abs(thumb_tip[0] - index_tip[0])
                    distance_y = abs(thumb_tip[1] - index_tip[1])
                    distance = math.hypot(distance_x, distance_y)
                    distance = max(min_distance, min(distance, max_distance))

                    volume_scalar = (distance - min_distance) / (max_distance - min_distance)

                    # ทำให้การเปลี่ยนเสียงนุ่มนวลขึ้น
                    smooth_factor = 0.2  
                    last_volume = (1 - smooth_factor) * last_volume + smooth_factor * volume_scalar

                    # ป้องกันการเปลี่ยนเสียงถ้าต่างกันน้อยเกินไป
                    if abs(volume.GetMasterVolumeLevelScalar() - last_volume) > 0.02:
                        volume.SetMasterVolumeLevelScalar(last_volume, None)

                    # วาดเส้นระหว่างนิ้วโป้งกับชี้
                    cv2.line(img, (thumb_tip[0], thumb_tip[1]), (index_tip[0], index_tip[1]), (0, 255, 0), 3)

                # แสดงระดับเสียง
                volume_percent = int(last_volume * 100)
                cv2.putText(img, f'Volume: {volume_percent}%', (50, 50),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

                # แสดงแถบระดับเสียง
                bar_height = int((1 - last_volume) * 300)
                bar_color = (0, 255, 0) if last_volume > 0.5 else (0, 0, 255)  # สีเขียว = ดัง, แดง = เบา
                cv2.rectangle(img, (50, 100), (100, 400), (255, 0, 0), 3)  
                cv2.rectangle(img, (50, 100 + bar_height), (100, 400), bar_color, -1)

                # แสดงไอคอนเสียง
                cv2.putText(img, "🔊" if last_volume > 0.5 else "🔈", (120, 380), 
                            cv2.FONT_HERSHEY_COMPLEX, 1, bar_color, 2)

            elif hand["type"] == "Left" and len(hand["lmList"]) >= 9:  # มือซ้าย -> ควบคุมวิดีโอ
                current_time = time.time()

                if current_time - last_time > 1:  # เช็คเวลาห่างกัน 1 วินาที
                    if fingers == [0, 1, 0, 0, 0]:  # ชี้นิ้วมือซ้ายไปทางขวา (เลื่อนคลิปไปข้างหน้า 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('right')  # เลื่อนคลิปไปข้างหน้า 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน

                    elif fingers == [0, 1, 1, 0, 0]:  # ชี้ 2 นิ้ว (นิ้วชี้, นิ้วกลาง) (ย้อนคลิป 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('left')  # เลื่อนคลิปย้อนกลับ 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน


                    # ไปคลิปถัดไป
                    elif fingers == [0, 1, 1, 1, 1]:  # ชี้ 4 นิ้ว (ชี้นิ้ว, นิ้วกลาง, นิ้วนาง, นิ้วก้อย)
                        if current_time - last_next_time > next_interval:  # 5 วินาที
                            pyautogui.hotkey('shift', 'n')  # ใช้ shift + n ไปคลิปถัดไป
                            last_next_time = current_time  # รีเซ็ตเวลาการไปคลิปถัดไป


                    elif fingers == [0, 0, 0, 0, 1]:  # กำมือ = หยุดวิดีโอ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

                    elif fingers == [1, 1, 1, 1, 1]:  # แบมือ 5 นิ้ว = เล่นวิดีโอต่อ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

    img = cv2.resize(img, (960, 720))
    cv2.imshow("image", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'comtypes'

In [39]:
 import cv2
from cvzone.HandTrackingModule import HandDetector
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import pyautogui  # ใช้สำหรับควบคุมวิดีโอ
import time

# ตั้งค่ากล้อง
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8)  # เพิ่มความแม่นยำในการตรวจจับ

# ตั้งค่าระบบเสียง
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# กำหนดค่าต่ำสุด-สูงสุดของระยะมือ
min_distance = 20
max_distance = 180
adjusting_volume = False  # เช็คว่ากำลังปรับเสียงอยู่หรือไม่
last_volume = volume.GetMasterVolumeLevelScalar()  # เก็บระดับเสียงล่าสุด

# ตัวแปรใหม่เช็คสถานะการกด
last_action = None
last_time = time.time()  # ใช้จับเวลาเพื่อป้องกันการกดซ้ำ
last_move_time = time.time()  # จับเวลาสำหรับการเลื่อนคลิป
last_skip_time = time.time()  # สำหรับจับเวลาเมื่อข้ามโฆษณา
last_next_time = time.time()  # สำหรับจับเวลาไปคลิปถัดไป

# ความเร็วในการเลื่อนคลิป
move_interval = 1  # 1 วินาทีในการเลื่อนคลิป
move_duration = 5  # 5 วินาทีในการเลื่อนคลิป
skip_interval = 5  # 5 วินาทีหลังจากข้ามโฆษณา
next_interval = 5  # 5 วินาทีหลังจากไปคลิปถัดไป

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)

    if hands:
        for hand in hands:
            fingers = detector.fingersUp(hand)  # ตรวจสอบนิ้วที่ยก

            if hand["type"] == "Right" and len(hand["lmList"]) >= 9:  # มือขวา -> ควบคุมเสียง
                if fingers == [1, 1, 1, 1, 1]:  # กางมือ 5 นิ้ว -> หยุดที่ระดับเสียงเดิม
                    adjusting_volume = False  

                elif fingers == [1, 1, 0, 0, 0]:  # ชูแค่โป้งกับชี้ -> ปรับเสียง
                    adjusting_volume = True
                    thumb_tip = hand["lmList"][4]
                    index_tip = hand["lmList"][8]
                    
                    # คำนวณระยะห่างระหว่างนิ้วโป้งและนิ้วชี้
                    distance_x = abs(thumb_tip[0] - index_tip[0])
                    distance_y = abs(thumb_tip[1] - index_tip[1])
                    distance = math.hypot(distance_x, distance_y)
                    distance = max(min_distance, min(distance, max_distance))

                    volume_scalar = (distance - min_distance) / (max_distance - min_distance)

                    # ทำให้การเปลี่ยนเสียงนุ่มนวลขึ้น
                    smooth_factor = 0.2  
                    last_volume = (1 - smooth_factor) * last_volume + smooth_factor * volume_scalar

                    # ป้องกันการเปลี่ยนเสียงถ้าต่างกันน้อยเกินไป
                    if abs(volume.GetMasterVolumeLevelScalar() - last_volume) > 0.02:
                        volume.SetMasterVolumeLevelScalar(last_volume, None)

                    # วาดเส้นระหว่างนิ้วโป้งกับชี้
                    cv2.line(img, (thumb_tip[0], thumb_tip[1]), (index_tip[0], index_tip[1]), (0, 255, 0), 3)

                # แสดงระดับเสียง
                volume_percent = int(last_volume * 100)
                cv2.putText(img, f'Volume: {volume_percent}%', (50, 50),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

                # แสดงแถบระดับเสียง
                bar_height = int((1 - last_volume) * 300)
                bar_color = (0, 255, 0) if last_volume > 0.5 else (0, 0, 255)  # สีเขียว = ดัง, แดง = เบา
                cv2.rectangle(img, (50, 100), (100, 400), (255, 0, 0), 3)  
                cv2.rectangle(img, (50, 100 + bar_height), (100, 400), bar_color, -1)

                # แสดงไอคอนเสียง
                cv2.putText(img, "🔊" if last_volume > 0.5 else "🔈", (120, 380), 
                            cv2.FONT_HERSHEY_COMPLEX, 1, bar_color, 2)

            elif hand["type"] == "Left" and len(hand["lmList"]) >= 9:  # มือซ้าย -> ควบคุมวิดีโอ
                current_time = time.time()

                if current_time - last_time > 1:  # เช็คเวลาห่างกัน 1 วินาที
                    if fingers == [0, 1, 0, 0, 0]:  # ชี้นิ้วมือซ้ายไปทางขวา (เลื่อนคลิปไปข้างหน้า 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('right')  # เลื่อนคลิปไปข้างหน้า 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน

                    elif fingers == [0, 1, 1, 0, 0]:  # ชี้ 2 นิ้ว (นิ้วชี้, นิ้วกลาง) (ย้อนคลิป 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('left')  # เลื่อนคลิปย้อนกลับ 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน


                    # ไปคลิปถัดไป
                    elif fingers == [0, 1, 1, 1, 1]:  # ชี้ 4 นิ้ว (ชี้นิ้ว, นิ้วกลาง, นิ้วนาง, นิ้วก้อย)
                        if current_time - last_next_time > next_interval:  # 5 วินาที
                            pyautogui.hotkey('shift', 'n')  # ใช้ shift + n ไปคลิปถัดไป
                            last_next_time = current_time  # รีเซ็ตเวลาการไปคลิปถัดไป


                    elif fingers == [0, 0, 0, 0, 1]:  # กำมือ = หยุดวิดีโอ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

                    elif fingers == [1, 1, 1, 1, 1]:  # แบมือ 5 นิ้ว = เล่นวิดีโอต่อ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

    img = cv2.resize(img, (960, 720))
    cv2.imshow("image", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'pycaw'

In [37]:
 import cv2
from cvzone.HandTrackingModule import HandDetector
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import pyautogui  # ใช้สำหรับควบคุมวิดีโอ
import time

# ตั้งค่ากล้อง
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8)  # เพิ่มความแม่นยำในการตรวจจับ

# ตั้งค่าระบบเสียง
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# กำหนดค่าต่ำสุด-สูงสุดของระยะมือ
min_distance = 20
max_distance = 180
adjusting_volume = False  # เช็คว่ากำลังปรับเสียงอยู่หรือไม่
last_volume = volume.GetMasterVolumeLevelScalar()  # เก็บระดับเสียงล่าสุด

# ตัวแปรใหม่เช็คสถานะการกด
last_action = None
last_time = time.time()  # ใช้จับเวลาเพื่อป้องกันการกดซ้ำ
last_move_time = time.time()  # จับเวลาสำหรับการเลื่อนคลิป
last_skip_time = time.time()  # สำหรับจับเวลาเมื่อข้ามโฆษณา
last_next_time = time.time()  # สำหรับจับเวลาไปคลิปถัดไป

# ความเร็วในการเลื่อนคลิป
move_interval = 1  # 1 วินาทีในการเลื่อนคลิป
move_duration = 5  # 5 วินาทีในการเลื่อนคลิป
skip_interval = 5  # 5 วินาทีหลังจากข้ามโฆษณา
next_interval = 5  # 5 วินาทีหลังจากไปคลิปถัดไป

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)

    if hands:
        for hand in hands:
            fingers = detector.fingersUp(hand)  # ตรวจสอบนิ้วที่ยก

            if hand["type"] == "Right" and len(hand["lmList"]) >= 9:  # มือขวา -> ควบคุมเสียง
                if fingers == [1, 1, 1, 1, 1]:  # กางมือ 5 นิ้ว -> หยุดที่ระดับเสียงเดิม
                    adjusting_volume = False  

                elif fingers == [1, 1, 0, 0, 0]:  # ชูแค่โป้งกับชี้ -> ปรับเสียง
                    adjusting_volume = True
                    thumb_tip = hand["lmList"][4]
                    index_tip = hand["lmList"][8]
                    
                    # คำนวณระยะห่างระหว่างนิ้วโป้งและนิ้วชี้
                    distance_x = abs(thumb_tip[0] - index_tip[0])
                    distance_y = abs(thumb_tip[1] - index_tip[1])
                    distance = math.hypot(distance_x, distance_y)
                    distance = max(min_distance, min(distance, max_distance))

                    volume_scalar = (distance - min_distance) / (max_distance - min_distance)

                    # ทำให้การเปลี่ยนเสียงนุ่มนวลขึ้น
                    smooth_factor = 0.2  
                    last_volume = (1 - smooth_factor) * last_volume + smooth_factor * volume_scalar

                    # ป้องกันการเปลี่ยนเสียงถ้าต่างกันน้อยเกินไป
                    if abs(volume.GetMasterVolumeLevelScalar() - last_volume) > 0.02:
                        volume.SetMasterVolumeLevelScalar(last_volume, None)

                    # วาดเส้นระหว่างนิ้วโป้งกับชี้
                    cv2.line(img, (thumb_tip[0], thumb_tip[1]), (index_tip[0], index_tip[1]), (0, 255, 0), 3)

                # แสดงระดับเสียง
                volume_percent = int(last_volume * 100)
                cv2.putText(img, f'Volume: {volume_percent}%', (50, 50),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

                # แสดงแถบระดับเสียง
                bar_height = int((1 - last_volume) * 300)
                bar_color = (0, 255, 0) if last_volume > 0.5 else (0, 0, 255)  # สีเขียว = ดัง, แดง = เบา
                cv2.rectangle(img, (50, 100), (100, 400), (255, 0, 0), 3)  
                cv2.rectangle(img, (50, 100 + bar_height), (100, 400), bar_color, -1)

                # แสดงไอคอนเสียง
                cv2.putText(img, "🔊" if last_volume > 0.5 else "🔈", (120, 380), 
                            cv2.FONT_HERSHEY_COMPLEX, 1, bar_color, 2)

            elif hand["type"] == "Left" and len(hand["lmList"]) >= 9:  # มือซ้าย -> ควบคุมวิดีโอ
                current_time = time.time()

                if current_time - last_time > 1:  # เช็คเวลาห่างกัน 1 วินาที
                    if fingers == [0, 1, 0, 0, 0]:  # ชี้นิ้วมือซ้ายไปทางขวา (เลื่อนคลิปไปข้างหน้า 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('right')  # เลื่อนคลิปไปข้างหน้า 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน

                    elif fingers == [0, 1, 1, 0, 0]:  # ชี้ 2 นิ้ว (นิ้วชี้, นิ้วกลาง) (ย้อนคลิป 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('left')  # เลื่อนคลิปย้อนกลับ 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน


                    # ไปคลิปถัดไป
                    elif fingers == [0, 1, 1, 1, 1]:  # ชี้ 4 นิ้ว (ชี้นิ้ว, นิ้วกลาง, นิ้วนาง, นิ้วก้อย)
                        if current_time - last_next_time > next_interval:  # 5 วินาที
                            pyautogui.hotkey('shift', 'n')  # ใช้ shift + n ไปคลิปถัดไป
                            last_next_time = current_time  # รีเซ็ตเวลาการไปคลิปถัดไป


                    elif fingers == [0, 0, 0, 0, 1]:  # กำมือ = หยุดวิดีโอ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

                    elif fingers == [1, 1, 1, 1, 1]:  # แบมือ 5 นิ้ว = เล่นวิดีโอต่อ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

    img = cv2.resize(img, (960, 720))
    cv2.imshow("image", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
 

In [15]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import pyautogui  # ใช้สำหรับควบคุมวิดีโอ
import time

# =======================
# ส่วนที่ 1: ตั้งค่ากล้อง
# =======================
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.9)  # เพิ่มความแม่นยำในการตรวจจับ

# =======================
# ส่วนที่ 2: ตั้งค่าระบบเสียง
# =======================
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# กำหนดค่าต่ำสุด-สูงสุดของระยะมือ
min_distance = 20
max_distance = 180
adjusting_volume = False  # เช็คว่ากำลังปรับเสียงอยู่หรือไม่
last_volume = volume.GetMasterVolumeLevelScalar()  # เก็บระดับเสียงล่าสุด

# =======================
# ส่วนที่ 3: ตัวแปรและการตั้งค่าเวลาต่างๆ
# =======================
last_action = None
last_time = time.time()  # ใช้จับเวลาเพื่อป้องกันการกดซ้ำ
last_move_time = time.time()  # จับเวลาสำหรับการเลื่อนคลิป
last_next_time = time.time()  # สำหรับจับเวลาไปคลิปถัดไป

# ความเร็วในการเลื่อนคลิป
move_interval = 1  # 1 วินาทีในการเลื่อนคลิป
next_interval = 5  # 5 วินาทีหลังจากไปคลิปถัดไป

# =======================
# ส่วนที่ 4: การอ่านข้อมูลจากกล้อง
# =======================
while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)

    if hands:
        for hand in hands:
            fingers = detector.fingersUp(hand)  # ตรวจสอบนิ้วที่ยก

            # =======================
            # ส่วนที่ 5: การควบคุมเสียงด้วยมือขวา
            # =======================
            if hand["type"] == "Right" and len(hand["lmList"]) >= 9:  # มือขวา -> ควบคุมเสียง
                if fingers == [1, 1, 1, 1, 1]:  # กางมือ 5 นิ้ว -> หยุดที่ระดับเสียงเดิม
                    adjusting_volume = False  

                elif fingers == [1, 1, 0, 0, 0]:  # ชูแค่โป้งกับชี้ -> ปรับเสียง
                    adjusting_volume = True
                    thumb_tip = hand["lmList"][4]
                    index_tip = hand["lmList"][8]
                    
                    # คำนวณระยะห่างระหว่างนิ้วโป้งและนิ้วชี้
                    distance_x = abs(thumb_tip[0] - index_tip[0])
                    distance_y = abs(thumb_tip[1] - index_tip[1])
                    distance = math.hypot(distance_x, distance_y)
                    distance = max(min_distance, min(distance, max_distance))

                    volume_scalar = (distance - min_distance) / (max_distance - min_distance)

                    # ทำให้การเปลี่ยนเสียงนุ่มนวลขึ้น
                    smooth_factor = 0.2  
                    last_volume = (1 - smooth_factor) * last_volume + smooth_factor * volume_scalar

                    # ป้องกันการเปลี่ยนเสียงถ้าต่างกันน้อยเกินไป
                    if abs(volume.GetMasterVolumeLevelScalar() - last_volume) > 0.02:
                        volume.SetMasterVolumeLevelScalar(last_volume, None)

                    # วาดเส้นระหว่างนิ้วโป้งกับชี้
                    cv2.line(img, (thumb_tip[0], thumb_tip[1]), (index_tip[0], index_tip[1]), (0, 255, 0), 3)

                # แสดงระดับเสียง
                volume_percent = int(last_volume * 100)
                cv2.putText(img, f'Volume: {volume_percent}%', (50, 50),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

                # แสดงแถบระดับเสียง
                bar_height = int((1 - last_volume) * 300)
                bar_color = (0, 255, 0) if last_volume > 0.5 else (0, 0, 255)  # สีเขียว = ดัง, แดง = เบา
                cv2.rectangle(img, (50, 100), (100, 400), (255, 0, 0), 3)  
                cv2.rectangle(img, (50, 100 + bar_height), (100, 400), bar_color, -1)

                # แสดงไอคอนเสียง
                cv2.putText(img, "🔊" if last_volume > 0.5 else "🔈", (120, 380), 
                            cv2.FONT_HERSHEY_COMPLEX, 1, bar_color, 2)

            # =======================
            # ส่วนที่ 6: การควบคุมวิดีโอด้วยมือซ้าย
            # =======================
            elif hand["type"] == "Left" and len(hand["lmList"]) >= 9:  # มือซ้าย -> ควบคุมวิดีโอ
                current_time = time.time()

                if current_time - last_time > 1:  # เช็คเวลาห่างกัน 1 วินาที
                    if fingers == [0, 1, 0, 0, 0]:  # ชี้นิ้วมือซ้ายไปทางขวา (เลื่อนคลิปไปข้างหน้า 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('right')  # เลื่อนคลิปไปข้างหน้า 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน

                    elif fingers == [0, 1, 1, 0, 0]:  # ชี้ 2 นิ้ว (นิ้วชี้, นิ้วกลาง) (ย้อนคลิป 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('left')  # เลื่อนคลิปย้อนกลับ 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน

                    # ไปคลิปถัดไป
                    elif fingers == [0, 1, 1, 1, 1]:  # ชี้ 4 นิ้ว (ชี้นิ้ว, นิ้วกลาง, นิ้วนาง, นิ้วก้อย)
                        if current_time - last_next_time > next_interval:  # 5 วินาที
                            pyautogui.hotkey('shift', 'n')  # ใช้ shift + n ไปคลิปถัดไป
                            last_next_time = current_time  # รีเซ็ตเวลาการไปคลิปถัดไป

                    elif fingers == [1, 1, 1, 1, 1]:  # แบมือ 5 นิ้ว = เล่นวิดีโอต่อ
                        pyautogui.press('space')  
                        last_time = current_time  # รีเซ็ตเวลา

    # =======================
    # ส่วนที่ 7: การแสดงผลภาพ 
    # =======================
    img = cv2.resize(img, (960, 720))
    cv2.imshow("image", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# =======================
# ส่วนที่ 8: ปิดกล้องและหน้าต่าง
# =======================
cap.release()
cv2.destroyAllWindows()


# 1. Business Understanding
Objective:
Develop an application that allows users to control volume and video playback using hand gestures, such as adjusting the volume, skipping clips, or pausing/playing videos.

Business Questions:
- Do users need a convenient way to control volume without touching their devices or using voice commands?
- Do users want an easy way to manage video playback using hand movements?
  
Goals:
- Use hand motion detection to adjust the volume by measuring the distance between the thumb and index finger.
- Utilize hand gestures to control video playback (pause, play, skip forward, rewind).

In [13]:
# วัตถุประสงค์ที่ตั้งไว้ในการพัฒนาแอป
import cv2
from cvzone.HandTrackingModule import HandDetector
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import pyautogui
import time

# การตั้งค่ากล้องและตัวตรวจจับมือ
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8)  # ใช้ตัวตรวจจับมือที่มีความแม่นยำสูง

# การควบคุมระดับเสียง
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))  # ควบคุมระดับเสียงจาก pycaw


# 2. Data Understanding
Data Sources:
- Camera Data: Captured using cv2.VideoCapture(0) to obtain images from the webcam.
- Hand Movement Data: Processed using the HandDetector module from cvzone to detect and analyze hand gestures.
- Volume Level Data: Managed using the pycaw library to control system volume.
- Video Playback Control Data: Handled via pyautogui to simulate keyboard inputs for skipping, pausing, or playing videos.
  
Data Preparation:
- The raw camera input will be processed to detect and recognize hand movements, enabling gesture-based control functionalities.

In [ ]:
# การใช้งานข้อมูลจากกล้องและการตรวจจับมือ
cap = cv2.VideoCapture(0)  # เปิดกล้องเพื่อจับภาพ
detector = HandDetector(detectionCon=0.8)  # ตั้งค่าความแม่นยำในการตรวจจับมือ

# ข้อมูลการควบคุมเสียงจาก pycaw
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))  # ควบคุมระดับเสียง


# 3. Data Preparation
Steps for Data Preparation:
- Use cv2.VideoCapture to capture data from the camera.
- Use HandDetector to extract hand movement data, such as finger coordinates.
- Calculate the distance between the thumb and index finger to control the volume level.
- Utilize hand movement data to manage video playback.

In [ ]:
# อ่านภาพจากกล้อง
success, img = cap.read()  # อ่านภาพจากกล้อง
hands, img = detector.findHands(img)  # ตรวจจับมือจากภาพ

# ตรวจจับมือและตรวจสอบท่าทางนิ้วมือ
if hands:  # ถ้ามีมือ
    for hand in hands:  # ตรวจจับมือแต่ละข้าง
        fingers = detector.fingersUp(hand)  # ตรวจสอบนิ้วที่ยก


# 4. Modeling
Model Used:
- No machine learning model is used; instead, hand detection and distance calculation are implemented.
  
- Volume Control: The volume level is adjusted based on the distance between the thumb and index finger.
- Video Playback Control: Hand gestures are used to manage video playback (skip, pause, play).

In [ ]:
# คำนวณระยะห่างระหว่างนิ้วโป้งและนิ้วชี้
thumb_tip = hand["lmList"][4]
index_tip = hand["lmList"][8]

distance_x = abs(thumb_tip[0] - index_tip[0])
distance_y = abs(thumb_tip[1] - index_tip[1])
distance = math.hypot(distance_x, distance_y)  # คำนวณระยะห่าง
distance = max(min_distance, min(distance, max_distance))

# คำนวณระดับเสียงจากระยะห่าง
volume_scalar = (distance - min_distance) / (max_distance - min_distance)


In [ ]:
 if current_time - last_time > 1:  # เช็คเวลาห่างกัน 1 วินาที
                    if fingers == [0, 1, 0, 0, 0]:  # ชี้นิ้วมือซ้ายไปทางขวา (เลื่อนคลิปไปข้างหน้า 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('right')  # เลื่อนคลิปไปข้างหน้า 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน

                    elif fingers == [0, 1, 1, 0, 0]:  # ชี้ 2 นิ้ว (นิ้วชี้, นิ้วกลาง) (ย้อนคลิป 5 วินาที)
                        if current_time - last_move_time > move_interval:
                            pyautogui.press('left')  # เลื่อนคลิปย้อนกลับ 5 วินาที
                            last_move_time = current_time  # รีเซ็ตเวลาการเลื่อน

                    # ไปคลิปถัดไป
                    elif fingers == [0, 1, 1, 1, 1]:  # ชี้ 4 นิ้ว (ชี้นิ้ว, นิ้วกลาง, นิ้วนาง, นิ้วก้อย)
                        if current_time - last_next_time > next_interval:  # 5 วินาที
                            pyautogui.hotkey('shift', 'n')  # ใช้ shift + n ไปคลิปถัดไป
                            last_next_time = current_time  # รีเซ็ตเวลาการไปคลิปถัดไป

# 5. Evaluation
Evaluation Criteria:
- Assess volume adjustment accuracy by detecting whether hand gestures correctly control the volume.
- Evaluate video playback control (skip, pause/play) to ensure expected functionality.

Assessment Methods:
- Verify that volume changes are neither too drastic nor insufficient.
- Check the accuracy of simulated key presses for video control.

In [ ]:
# ป้องกันการเปลี่ยนแปลงระดับเสียงถ้าการเปลี่ยนแปลงไม่มากพอ
if abs(volume.GetMasterVolumeLevelScalar() - last_volume) > 0.02:
    volume.SetMasterVolumeLevelScalar(last_volume, None)

# ควบคุมการเล่นวิดีโอ (เลื่อนคลิป)
if fingers == [0, 1, 0, 0, 0]:  # เลื่อนคลิปไปข้างหน้า
    pyautogui.press('right')  # ควบคุมการเล่นวิดีโอ


# 6. Deployment
Real-World Application:
- The code can be implemented on computers with a compatible camera and software for controlling audio and video.
- Users can conveniently control volume and video playback without using a mouse or keyboard.

Installation:
- Users need to install the required libraries, such as cv2, pycaw, pyautogui, and comtypes, before use.

In [ ]:
# ติดตั้งไลบรารีที่จำเป็น
pip install opencv-python cvzone pycaw pyautogui comtypes


Maintenance:
- Ensure proper lighting conditions to improve hand detection accuracy.
- Enhance volume control for smoother adjustments.

In [ ]:
# การทำให้การเปลี่ยนแปลงเสียงนุ่มนวล
smooth_factor = 0.2
last_volume = (1 - smooth_factor) * last_volume + smooth_factor * volume_scalar  # การเปลี่ยนเสียงที่เรียบเนียนขึ้น
